两个常见的操作是**索引和分配到索引位置**。无论列表有多大，这两个操作都需要相同的时间。当这样的操作和列表的大小无关时，它们$O(1)$ 。

另一个非常常见的编程任务是**加长列表**。有两种方法可以创建更长的列表，可以使用 append 方法或拼接运算符。append 方法是 $O(1)$。 然而，拼接运算符是 $O(k)$，其中 k 是要拼接的列表的大小。

让我们看看四种不同的方式，我们可以生成一个从0开始的n个数字的列表。首先，我们将尝试一个 for 循环并通过创建列表，然后我们将使用 append 而不是拼接。接下来，我们使用列表推导式创建列表，最后，也是最明显的方式，通过调用列表构造函数包装 range 函数。



In [1]:
def test1():
    l = []
    for i in range(1000):
        l = l + [i]

def test2():
    l = []
    for i in range(1000):
        l.append(i)

def test3():
    l = [i for i in range(1000)]

def test4():
    l = list(range(1000))

要捕获我们的每个函数执行所需的时间，我们将使用 Python 的 timeit 模块。timeit 模块旨在允许 Python 开发人员通过在一致的环境中运行函数并使用尽可能相似的操作系统的时序机制来进行跨平台时序测量。

要使用 timeit，你需要创建一个 Timer 对象，其参数是两个 Python 语句。第一个参数是一个你想要执行时间的 Python 语句; 第二个参数是一个将运行一次以设置测试的语句。然后 timeit 模块将计算执行语句所需的时间。默认情况下，timeit 将尝试运行语句一百万次。 当它完成时，它返回时间作为表示总秒数的浮点值。由于它执行语句一百万次，可以读取结果作为执行测试一次的微秒数。你还可以传递 timeit 一个参数名字为 number，允许你指定执行测试语句的次数。以下显示了运行我们的每个测试功能 1000 次需要多长时间。

In [2]:
from timeit import Timer

t1 = Timer("test1()", "from __main__ import test1")
print("concat ",t1.timeit(number=1000), "milliseconds")
t2 = Timer("test2()", "from __main__ import test2")
print("append ",t2.timeit(number=1000), "milliseconds")
t3 = Timer("test3()", "from __main__ import test3")
print("comprehension ",t3.timeit(number=1000), "milliseconds")
t4 = Timer("test4()", "from __main__ import test4")
print("list range ",t4.timeit(number=1000), "milliseconds")

concat  1.492609306002123 milliseconds
append  0.09577773499768227 milliseconds
comprehension  0.03827453200210584 milliseconds
list range  0.0182234480016632 milliseconds


在上面的例子中，我们对 test1(), test2() 等的函数调用计时，setup 语句可能看起来很奇怪，所以我们详细说明下。你可能非常熟悉 from ,import 语句，但这通常用在 python 程序文件的开头。在这种情况下，from __main__ import test1 从 __main__ 命名空间导入到 timeit 设置的命名空间中。timeit 这么做是因为它想在一个干净的环境中做测试，而不会因为可能有你创建的任何杂变量，以一种不可预见的方式干扰你函数的性能。

从上面的试验清楚的看出，append 操作比拼接快得多。其他两种方法，列表生成器的速度是 append 的两倍。

最后一点，你上面看到的时间都是包括实际调用函数的一些开销，但我们可以假设函数调用开销在四种情况下是相同的，所以我们仍然得到的是有意义的比较。因此，拼接字符串操作需要 6.54 毫秒并不准确，而是拼接字符串这个函数需要 6.54 毫秒。你可以测试调用空函数所需要的时间，并从上面的数字中减去它。

现在我们已经看到了如何具体测试性能，见 表1., 你可能想知道 pop 两个不同的时间。当列表末尾调用 pop 时，它需要 $O(1)$, 但是当在列表中第一个元素或者中间任何地方调用 pop, 它是 $O(n)$。原因在于 Python 实现列表的方式，当一个项从列表前面取出，列表中的其他元素靠近起始位置移动一个位置。你会看到索引操作为 $O(1)$。python的实现者会权衡选择一个好的方案。

<center>表1. 列表操作大O表示</center>

操作|大O表示
-|-
index []|O(1)
index assignment|O(1)
append|O(1)
pop()|O(1)
pop(i)|O(n)
insert(i,item)|O(n)
del operator|O(n)
iteration|O(n)
contains (in)|O(n)
get slice [x:y]|O(k)
del slice|O(n)
set slice|O(n+k)
reverse|O(n)
concatenate|O(k)
sort|O(n log n)
multiply|O(nk)

作为一种演示性能差异的方法，我们用 timeit 来做一个实验。我们的目标是验证列表从末尾 pop 元素和从开始 pop 元素的性能。同样，我们也想测量不同列表大小对这个时间的影响。我们期望看到的是，从列表末尾处弹出所需时间将保持不变，即使列表不断增长。而从列表开始处弹出元素时间将随列表增长而增加。

下面展示了两种 pop 方式的比较。从第一个示例看出，从末尾弹出需要 0.0001 毫秒。从开始弹出要花费 1.73 毫秒。对于一个 200 万的元素列表，相差 17000 倍。

需要注意的几点，第一， from __main__ import x , 虽然我们没有定义一个函数，我们确实希望能够在我们的测试中使用列表对象 x, 这种方法允许我们只计算单个弹出语句，获得该操作最精确的测量时间。因为 timer 重复了 1000 次，该列表每次循环大小都减 1。但是由于初始列表大小为 200万，我们只减少总体大小的 0.05%。

In [4]:
popzero = Timer("x.pop(0)", "from __main__ import x")
popend = Timer("x.pop()", "from __main__ import x")

x = list(range(2000000))
popzero.timeit(number=1000)

1.7360390799985908

In [5]:
x = list(range(2000000))
popend.timeit(number=1000)

0.00012579300164361484

虽然我们第一个测试显示 pop(0) 比 pop() 慢， 但它没有证明 pop(0) 是 $O(n)$, pop()是 $O(1)$。要验证它，我们需要看下一系列列表大小的调用效果。

In [6]:
popzero = Timer("x.pop(0)", "from __main__ import x")
popend = Timer("x.pop()", "from __main__ import x")
print("pop(0)   pop()")
for i in range(1000000,100000001,1000000):
    x = list(range(i))
    pt = popend.timeit(number=1000)
    x = list(range(i))
    pz = popzero.timeit(number=1000)
    print("%15.5f, %15.5f" %(pz,pt))

pop(0)   pop()
        0.42726,         0.00012
        1.72859,         0.00012
        2.91645,         0.00012
        4.03595,         0.00011
        5.11345,         0.00011
        6.16898,         0.00011
        7.15000,         0.00011
        7.92470,         0.00011
        9.00423,         0.00011
        9.97775,         0.00012
       11.31846,         0.00011
       12.00593,         0.00011
       13.09338,         0.00012
       14.12397,         0.00013
       14.83645,         0.00012
       16.12087,         0.00011
       16.62969,         0.00011
       18.15409,         0.00011
       19.29647,         0.00011
       20.13299,         0.00011


KeyboardInterrupt: 

![pop 与 pop(0) 操作性能的比较](pic/2.6.1.png)
<center>图1. pop 与 pop(0) 操作性能的比较]</center>

图1展示了我们实验的结果，你可以看到，随着列表变长，pop(0) 时间也增加，而 pop() 时间保持非常平坦。这正是我们期望看到的 $O(n)$ 和 $O(1)$。